## परिचय

यस पाठले समेट्नेछ:  
- के हो फंक्शन कलिङ र यसको प्रयोगका केसहरू  
- OpenAI प्रयोग गरेर फंक्शन कल कसरी सिर्जना गर्ने  
- फंक्शन कललाई अनुप्रयोगमा कसरी एकीकृत गर्ने  

## सिकाइ लक्ष्यहरू

यस पाठ पूरा गरेपछि तपाईं जान्नुहुनेछ र बुझ्नुहुनेछ:  

- फंक्शन कलिङ प्रयोग गर्ने उद्देश्य  
- OpenAI सेवा प्रयोग गरेर फंक्शन कल सेटअप गर्ने  
- तपाईंको अनुप्रयोगको प्रयोग केसका लागि प्रभावकारी फंक्शन कल डिजाइन गर्ने


## फंक्शन कलहरू बुझ्न

यस पाठका लागि, हामी हाम्रो शिक्षा स्टार्टअपका लागि एउटा सुविधा निर्माण गर्न चाहन्छौं जसले प्रयोगकर्ताहरूलाई टेक्निकल कोर्सहरू फेला पार्न च्याटबोट प्रयोग गर्न अनुमति दिन्छ। हामी उनीहरूको सीप स्तर, वर्तमान भूमिका र रुचिको प्रविधि अनुसार कोर्सहरू सिफारिस गर्नेछौं।

यसलाई पूरा गर्न हामी निम्न संयोजन प्रयोग गर्नेछौं:
 - `OpenAI` प्रयोगकर्ताका लागि च्याट अनुभव सिर्जना गर्न
 - `Microsoft Learn Catalog API` प्रयोगकर्ताहरूलाई उनीहरूको अनुरोध अनुसार कोर्सहरू फेला पार्न मद्दत गर्न
 - `Function Calling` प्रयोगकर्ताको सोधपुछ लिएर API अनुरोध गर्न फंक्शनमा पठाउन।

सुरु गर्नका लागि, हामीले किन फंक्शन कल प्रयोग गर्न चाहन्छौं भन्ने कुरा हेरौं:

print("अर्को अनुरोधमा सन्देशहरू:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # GPT बाट नयाँ प्रतिक्रिया प्राप्त गर्नुहोस् जहाँ यसले फंक्शन प्रतिक्रिया देख्न सक्छ

print(second_response.choices[0].message)


### किन Function Calling

यदि तपाईंले यस कोर्समा कुनै अन्य पाठ पूरा गर्नुभएको छ भने, तपाईंले Large Language Models (LLMs) को प्रयोग गर्ने शक्ति बुझ्नुभएको होला। आशा छ तपाईंले तिनीहरूको केही सीमितताहरू पनि देख्नुभएको छ।

Function Calling OpenAI सेवा को एउटा सुविधा हो जुन निम्न चुनौतीहरू समाधान गर्न डिजाइन गरिएको छ:

असंगत प्रतिक्रिया ढाँचा:
- Function calling अघि, ठूलो भाषा मोडेलबाट प्राप्त प्रतिक्रियाहरू असंरचित र असंगत हुन्थे। विकासकर्ताहरूले प्रत्येक भिन्नतालाई सम्हाल्न जटिल प्रमाणीकरण कोड लेख्नुपर्थ्यो।

बाह्य डाटासँग सीमित एकीकरण:
- यस सुविधाअघि, अनुप्रयोगका अन्य भागहरूबाट डाटा च्याट सन्दर्भमा समावेश गर्न गाह्रो हुन्थ्यो।

प्रतिक्रिया ढाँचाहरूलाई मानकीकृत गरेर र बाह्य डाटासँग सहज एकीकरण सक्षम पारेर, function calling विकासलाई सरल बनाउँछ र अतिरिक्त प्रमाणीकरण तर्कको आवश्यकता घटाउँछ।

प्रयोगकर्ताहरूले "स्टकहोममा हालको मौसम कस्तो छ?" जस्ता उत्तरहरू प्राप्त गर्न सक्दैनथे। यसको कारण मोडेलहरूलाई तालिम दिइएको डाटाको समयसम्म सीमित गरिएको थियो।

तलको उदाहरणले यो समस्या देखाउँछ:

मानौं हामी विद्यार्थीहरूको डाटाबेस बनाउन चाहन्छौं ताकि उनीहरूलाई सही कोर्स सिफारिस गर्न सकियोस्। तल हामीसँग दुई विद्यार्थीहरूको विवरण छ जुन डाटामा धेरै समान छन्।


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

हामी यसलाई डेटा पार्स गर्न LLM मा पठाउन चाहन्छौं। यसलाई पछि हाम्रो अनुप्रयोगमा API मा पठाउन वा डेटाबेसमा भण्डारण गर्न प्रयोग गर्न सकिन्छ।

हामी दुई समान प्रॉम्प्टहरू सिर्जना गरौं जसमा हामी LLM लाई कुन जानकारीमा हामी रुचि राख्छौं भनेर निर्देशन दिन्छौं:


हामी यसलाई हाम्रो उत्पादनका लागि महत्वपूर्ण भागहरू विश्लेषण गर्न LLM मा पठाउन चाहन्छौं। त्यसैले हामी LLM लाई निर्देशन दिन दुई समान प्रॉम्प्टहरू बनाउन सक्छौं:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


यी दुई प्रॉम्प्टहरू सिर्जना गरेपछि, हामी तिनीहरूलाई `openai.ChatCompletion` प्रयोग गरेर LLM मा पठाउनेछौं। हामी प्रॉम्प्टलाई `messages` भेरिएबलमा भण्डारण गर्छौं र रोललाई `user` मा असाइन गर्छौं। यो प्रयोगकर्ताबाट च्याटबोटमा पठाइएको सन्देशको नक्कल गर्न हो।


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

अब हामी दुवै अनुरोधहरू LLM मा पठाउन सक्छौं र प्राप्त प्रतिक्रिया जाँच गर्न सक्छौं।


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

यद्यपि प्रॉम्प्टहरू उस्तै छन् र विवरणहरू समान छन्, हामी `Grades` सम्पत्तिको फरक-फरक ढाँचाहरू प्राप्त गर्न सक्छौं।

यदि तपाईं माथिको सेललाई धेरै पटक चलाउनुहुन्छ भने, ढाँचा `3.7` वा `3.7 GPA` हुन सक्छ।

यो किनभने LLM ले लेखिएको प्रॉम्प्टको रूपमा असंरचित डाटा लिन्छ र असंरचित डाटा नै फर्काउँछ। हामीसँग संरचित ढाँचा हुन आवश्यक छ ताकि हामीलाई थाहा होस् कि यो डाटा भण्डारण गर्दा वा प्रयोग गर्दा के अपेक्षा गर्ने।

कार्यात्मक कलिङ प्रयोग गरेर, हामी सुनिश्चित गर्न सक्छौं कि हामीलाई संरचित डाटा फिर्ता प्राप्त हुन्छ। कार्यात्मक कलिङ प्रयोग गर्दा, LLM ले वास्तवमा कुनै पनि फंक्शन कल वा चलाउँदैन। यसको सट्टा, हामी LLM लाई यसको प्रतिक्रियाहरूका लागि पालना गर्न संरचना सिर्जना गर्छौं। त्यसपछि हामी ती संरचित प्रतिक्रियाहरूलाई हाम्रो अनुप्रयोगहरूमा कुन फंक्शन चलाउने थाहा पाउन प्रयोग गर्छौं।


![Function Calling Flow Diagram](../../../../translated_images/Function-Flow.083875364af4f4bb.ne.png)


हामी त्यसपछि फंक्शनबाट फर्काइएको कुरा लिएर यसलाई LLM मा फिर्ता पठाउन सक्छौं। LLM त्यसपछि प्रयोगकर्ताको प्रश्नको उत्तर दिन प्राकृतिक भाषामा प्रतिक्रिया दिनेछ।


### कार्य कलहरू प्रयोग गर्ने केसहरू

**बाह्य उपकरणहरू कल गर्ने**  
च्याटबोटहरू प्रयोगकर्ताहरूका प्रश्नहरूको उत्तर दिनमा उत्कृष्ट हुन्छन्। कार्य कल प्रयोग गरेर, च्याटबोटहरूले प्रयोगकर्ताहरूका सन्देशहरूलाई केही कार्यहरू पूरा गर्न प्रयोग गर्न सक्छन्। उदाहरणका लागि, एक विद्यार्थीले च्याटबोटलाई "मेरो शिक्षकलाई इमेल पठाउनुहोस् भनी भन्नुहोस् कि मलाई यस विषयमा थप सहयोग चाहिन्छ" भन्न सक्छ। यसले `send_email(to: string, body: string)` नामक कार्य कल गर्न सक्छ।

**API वा डाटाबेस क्वेरीहरू सिर्जना गर्ने**  
प्रयोगकर्ताहरू प्राकृतिक भाषामा जानकारी खोज्न सक्छन् जुन एक फर्म्याट गरिएको क्वेरी वा API अनुरोधमा रूपान्तरण हुन्छ। यसको उदाहरणको रूपमा, एक शिक्षकले "अन्तिम असाइनमेन्ट पूरा गरेका विद्यार्थीहरू को हुन्?" भनेर सोध्न सक्छ जसले `get_completed(student_name: string, assignment: int, current_status: string)` नामक कार्य कल गर्न सक्छ।

**संरचित डाटा सिर्जना गर्ने**  
प्रयोगकर्ताहरूले एउटा पाठ ब्लक वा CSV लिएर LLM प्रयोग गरी त्यसबाट महत्वपूर्ण जानकारी निकाल्न सक्छन्। उदाहरणका लागि, एक विद्यार्थीले शान्ति सम्झौताहरूको बारेमा विकिपिडिया लेखलाई AI फ्ल्यास कार्डहरू बनाउन रूपान्तरण गर्न सक्छ। यसका लागि `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` नामक कार्य कल प्रयोग गर्न सकिन्छ।


## 2. तपाईंको पहिलो फंक्शन कल सिर्जना गर्दै 

फंक्शन कल सिर्जना गर्ने प्रक्रिया ३ मुख्य चरणहरू समावेश गर्दछ:  
१. तपाईंका फंक्शनहरूको सूची र प्रयोगकर्ताको सन्देश सहित Chat Completions API कल गर्नुहोस्  
२. मोडेलको प्रतिक्रिया पढ्नुहोस् र कार्य गर्न जस्तै फंक्शन वा API कल कार्यान्वयन गर्नुहोस्  
३. तपाईंको फंक्शनबाट प्राप्त प्रतिक्रियासहित फेरि Chat Completions API कल गर्नुहोस् र त्यो जानकारी प्रयोग गरी प्रयोगकर्तालाई प्रतिक्रिया सिर्जना गर्नुहोस्।


![फंक्शन कलको प्रवाह](../../../../translated_images/LLM-Flow.3285ed8caf4796d7.ne.png)


### फंक्शन कलका तत्वहरू

#### प्रयोगकर्ताको इनपुट

पहिलो चरण प्रयोगकर्ताको सन्देश सिर्जना गर्नु हो। यो गतिशील रूपमा टेक्स्ट इनपुटको मान लिएर असाइन गर्न सकिन्छ वा तपाईं यहाँ मान असाइन गर्न सक्नुहुन्छ। यदि यो तपाईंको पहिलो पटक हो Chat Completions API सँग काम गर्दा, हामीले सन्देशको `role` र `content` परिभाषित गर्न आवश्यक छ।

`role` हुन सक्छ `system` (नियमहरू सिर्जना गर्दै), `assistant` (मोडेल) वा `user` (अन्तिम प्रयोगकर्ता)। फंक्शन कलिङका लागि, हामी यसलाई `user` को रूपमा असाइन गर्नेछौं र एउटा उदाहरण प्रश्न दिनेछौं।


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### कार्यहरू सिर्जना गर्दै। 

अर्को हामीले एउटा कार्य र त्यस कार्यका प्यारामिटरहरू परिभाषित गर्नेछौं। यहाँ हामीले `search_courses` नामक एउटा मात्र कार्य प्रयोग गर्नेछौं तर तपाईंले धेरै कार्यहरू सिर्जना गर्न सक्नुहुन्छ।

**महत्त्वपूर्ण** : कार्यहरू LLM लाई सिस्टम सन्देशमा समावेश गरिन्छ र तपाईंको उपलब्ध टोकनहरूको मात्रामा समावेश गरिनेछ।


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**परिभाषाहरू** 

फङ्क्शन परिभाषा संरचनामा धेरै तहहरू हुन्छन्, प्रत्येकसँग आफ्नै गुणहरू हुन्छन्। यहाँ नेस्टेड संरचनाको एक विबरण छ:

**शीर्ष स्तर फङ्क्शन गुणहरू:**

`name` -  फङ्क्शनको नाम जुन हामी कल गर्न चाहन्छौं। 

`description` - यो फङ्क्शन कसरी काम गर्छ भन्ने विवरण हो। यहाँ स्पष्ट र विशिष्ट हुनु महत्त्वपूर्ण छ 

`parameters` - मानहरूको सूची र ढाँचा जुन तपाईं मोडेलले आफ्नो प्रतिक्रिया मा उत्पादन गर्न चाहनुहुन्छ 

**प्यारामिटर वस्तुका गुणहरू:**

`type` - प्यारामिटर वस्तुको डाटा प्रकार (सामान्यतया "object")

`properties` - विशिष्ट मानहरूको सूची जुन मोडेलले आफ्नो प्रतिक्रियाको लागि प्रयोग गर्नेछ 

**व्यक्तिगत प्यारामिटर गुणहरू:**

`name` - प्रोपर्टी कुञ्जी द्वारा अप्रत्यक्ष रूपमा परिभाषित (जस्तै, "role", "product", "level")

`type` - यस विशेष प्यारामिटरको डाटा प्रकार (जस्तै, "string", "number", "boolean") 

`description` - विशेष प्यारामिटरको विवरण 

**वैकल्पिक गुणहरू:**

`required` - एउटा एर्रे जसले कुन कुन प्यारामिटरहरू फङ्क्शन कल पूरा गर्न आवश्यक छन् भनेर सूचीबद्ध गर्छ 


### फंक्शन कल बनाउँदै 
फंक्शन परिभाषित गरेपछि, अब हामीले यसलाई Chat Completion API कलमा समावेश गर्न आवश्यक छ। हामीले यो अनुरोधमा `functions` थपेर गर्छौं। यस अवस्थामा `functions=functions` हुन्छ। 

`function_call` लाई `auto` मा सेट गर्ने विकल्प पनि छ। यसको अर्थ हामीले LLM लाई प्रयोगकर्ता सन्देशको आधारमा कुन फंक्शन कल गर्नुपर्छ भनेर निर्णय गर्न दिनेछौं, हामी आफैंले निर्धारण नगर्ने।


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

अब हामी प्रतिक्रिया हेर्न जाऔं र यसले कसरी ढाँचाबद्ध गरिएको छ हेर्नुहोस्:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

तपाईंले देख्न सक्नुहुन्छ कि कार्यको नाम कल गरिएको छ र प्रयोगकर्ता सन्देशबाट, LLM ले कार्यका तर्कहरू मिलाउन डेटा फेला पार्न सक्षम भयो।


## 3.एप्लिकेशनमा फंक्शन कलहरू एकीकृत गर्दै। 


हामीले LLM बाट प्राप्त गरिएको फर्म्याट गरिएको प्रतिक्रिया परीक्षण गरेपछि, अब हामी यसलाई एप्लिकेशनमा एकीकृत गर्न सक्छौं। 

### प्रवाह व्यवस्थापन 

यसलाई हाम्रो एप्लिकेशनमा एकीकृत गर्न, हामीले निम्न चरणहरू लिनु पर्छ: 

पहिले, OpenAI सेवाहरूमा कल गरौं र सन्देशलाई `response_message` नामक भेरिएबलमा भण्डारण गरौं।


In [ ]:
response_message = response.choices[0].message

अब हामी त्यो फङ्क्शन परिभाषित गर्नेछौं जसले Microsoft Learn API लाई कल गरेर कोर्सहरूको सूची प्राप्त गर्नेछ:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



एक उत्कृष्ट अभ्यासको रूपमा, हामी त्यसपछि हेर्नेछौं कि मोडेलले कुनै फंक्शन कल गर्न चाहन्छ कि छैन। त्यसपछि, हामी उपलब्ध फंक्शनहरू मध्ये एक सिर्जना गर्नेछौं र त्यसलाई कल भइरहेको फंक्शनसँग मिलाउनेछौं।  
हामी त्यसपछि फंक्शनका आर्गुमेन्टहरू लिनेछौं र तिनीहरूलाई LLM का आर्गुमेन्टहरूसँग मिलाउनेछौं।  

अन्तमा, हामी फंक्शन कल सन्देश र `search_courses` सन्देशले फर्काएका मानहरू थप्नेछौं। यसले LLM लाई प्रयोगकर्तालाई प्राकृतिक भाषामा प्रतिक्रिया दिन आवश्यक सबै जानकारी दिन्छ।


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



अब हामी अपडेट गरिएको सन्देश LLM लाई पठाउनेछौं ताकि हामीले API JSON ढाँचामा नभई प्राकृतिक भाषा प्रतिक्रिया प्राप्त गर्न सकौं।


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## कोड चुनौती

शानदार काम! OpenAI Function Calling को तपाईंको सिकाइलाई जारी राख्न तपाईं निम्न बनाउन सक्नुहुन्छ: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - फंक्शनका थप प्यारामिटरहरू जुन सिक्नेहरूलाई थप कोर्सहरू फेला पार्न मद्दत गर्न सक्छ। तपाईं उपलब्ध API प्यारामिटरहरू यहाँ फेला पार्न सक्नुहुन्छ:  
 - अर्को फंक्शन कल सिर्जना गर्नुहोस् जसले सिक्ने व्यक्तिको मातृभाषा जस्ता थप जानकारी लिन्छ  
 - फंक्शन कल र/वा API कलले कुनै उपयुक्त कोर्सहरू फिर्ता नगरेमा त्रुटि ह्यान्डलिङ सिर्जना गर्नुहोस्  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:
यो दस्तावेज AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी शुद्धताका लागि प्रयासरत छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटि वा अशुद्धता हुन सक्छ। मूल दस्तावेज यसको मूल भाषामा आधिकारिक स्रोत मानिनु पर्छ। महत्वपूर्ण जानकारीका लागि व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार छैनौं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
